In [ ]:
import os

# Input params
connection_string = os.environ['PG_CONNECTION_STRING']
delay = os.environ['DELAY']
frame = os.environ['FRAME']
update_carpool_status = os.environ['UPDATE_CARPOOL_STATUS'] or False
osrm_url = os.environ['OSRM_URL']

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

query = f"""
SELECT cc._id, operator_id, cc.datetime, cc.duration, cc.distance, cc.identity_id, cc.operator_journey_id, start_geo_code, end_geo_code, 
cc.trip_id,
cc.is_driver,
cc.operator_class,
ci.operator_user_id,
CASE 
      WHEN ci.phone_trunc IS NULL THEN left(ci.phone, -2)
      ELSE ci.phone_trunc
      END AS phone_trunc,
ST_X(ST_AsText(cc.start_position)) start_x,
ST_Y(ST_AsText(cc.start_position)) start_y, 
ST_X(ST_AsText(cc.end_position)) end_x,
ST_Y(ST_AsText(cc.end_position)) end_y, 
gmap_url(cc.start_position, cc.end_position)
FROM CARPOOL.CARPOOLS CC
JOIN carpool.identities ci on cc.identity_id = ci._id
 join geo.perimeters gps on cc.start_geo_code = gps.arr and gps.year = 2023 -- and gps.aom = '287500078'
 join geo.perimeters gpe on cc.end_geo_code = gpe.arr and gpe.year = 2023 -- and gpe.aom = '287500078'
JOIN policy.incentives pi on pi.carpool_id = cc._id -- and pi.policy_id = 459 
WHERE CC.DATETIME >= NOW() - '{delay} hours'::interval - '{frame} hours'::interval
	AND CC.DATETIME < NOW() - '{delay} hours'::interval and
      cc.is_driver = true 
"""

with engine.connect() as conn:
    df_carpool = pd.read_sql_query(text(query), conn)


In [ ]:

import requests

def get_osrm_data(x):
  response =  requests.get(f"{osrm_url}/route/v1/car/{x.start_x},{x.start_y};{x.end_x},{x.end_y}")
  body = response.json()
  x['osrm_duration'] = body['routes'][0]['duration']
  x['osrm_distance'] = body['routes'][0]['distance']
  return x

df_carpool = df_carpool.apply(get_osrm_data, axis = 1)
# df_carpool = df_carpool.assign(duration_delta=lambda x: x.duration * 100 / x.osrm_duration)
# df_carpool = df_carpool.assign(distance_delta=lambda x: x.distance * 100 / x.osrm_distance)


In [ ]:
operator_class_c_mask = df_carpool.operator_class == 'C'
df_only_class_c_trip = df_carpool[operator_class_c_mask]

Sur les trajets de class C

https://doc.covoiturage.beta.gouv.fr/nos-services/le-registre-de-preuve-de-covoiturage/quest-ce-que-cest/classes-de-preuve-and-identite/classes-a-b-c

# Relever les trajets : 
- Qui ont une distance estimée par osrm ou transmises inférieure à 300m.
- Qui ont une durée estimée par osrm ou transmises inférieures à 1 minutes.
- Qui ont une durée estimée par osrm supérieure à 2.5 fois le temps transmis. Exemple: temps estimé 25 minutes vs temps transmis 10 minutes
- Qui ont une distance estimée par osrm supérieure à 2.5 fois la distance transmise. Exemple: distance 1km sur une distance estimée 2.5km
- Qui ont une distance transmise supérieur à 4 fois la distance estimée osrm. 5 estimé 10 transmises (détours par rapport au plus court chemin possible) 
- Qui ont une durée transmise supérieure à 7 fois la durée estimée (embouteillage possible)

In [ ]:
less_than_300_meters_mask = (df_only_class_c_trip.distance < 300) | (df_only_class_c_trip.osrm_distance < 300)
less_than_1_minutes_mask = (df_only_class_c_trip.duration < 60) | (df_only_class_c_trip.osrm_duration < 60)
estimate_vs_computed_percent_more_duration_mask = (df_only_class_c_trip.osrm_duration > df_only_class_c_trip.duration * 2.5) | (df_only_class_c_trip.osrm_duration * 7 < df_only_class_c_trip.duration)
estimate_vs_computed_percent_more_distance_mask = (df_only_class_c_trip.osrm_distance > df_only_class_c_trip.distance * 2.5) | (df_only_class_c_trip.osrm_distance * 4 < df_only_class_c_trip.distance)

df_result = df_only_class_c_trip[less_than_300_meters_mask | less_than_1_minutes_mask | estimate_vs_computed_percent_more_duration_mask | estimate_vs_computed_percent_more_distance_mask]


In [ ]:
df_labels = df_result[['_id']]
df_labels = df_labels.assign(label='distance_duration_anomaly')
df_labels.rename(columns={"_id": "carpool_id"})

In [ ]:
import sqlalchemy as sa

if update_carpool_status is True:

    metadata = sa.MetaData(schema='carpool')
    metadata.reflect(bind=engine)

    table = metadata.tables['carpool.carpools']
    
    where_clause = table.c._id.in_(df_labels['carpool_id'].to_list())

    update_stmt = sa.update(table).where(where_clause).values(status='anomaly_error')

    with engine.connect() as conn:
        result = conn.execute(update_stmt)
        conn.commit()

In [ ]:
from sqlalchemy.dialects.postgresql import insert

def insert_or_do_nothing_on_conflict(table, conn, keys, data_iter):
    insert_stmt = insert(table.table).values(list(data_iter))
    on_duplicate_key_stmt = insert_stmt.on_conflict_do_nothing(index_elements=['carpool_id', 'label'])
    conn.execute(on_duplicate_key_stmt)

df_labels.to_sql(
    name="labels",
    schema="anomaly",
    con=engine,
    if_exists="append",
    index=False,
    method=insert_or_do_nothing_on_conflict
)

In [ ]:
# df_result_light = df_result[['_id','operator_id','gmap_url', 'distance', 'duration', 'osrm_distance', 'osrm_duration', 'distance_delta', 'duration_delta']]
# df_result_light['duration'] = df_result_light['duration']/60
# df_result_light['osrm_duration'] = df_result_light['osrm_duration']/60 
# df_result_light['duration_delta'] = df_result_light['duration_delta']/60 

Regarder le restes des trajets pour voir si il n'y a pas d'autre anomalie possible

In [ ]:
# df_row_check = df_only_class_c_trip[~df_only_class_c_trip._id.isin(df_result['_id'])]

# df_row_check = df_row_check[['_id','operator_id','gmap_url', 'distance', 'duration', 'osrm_distance', 'osrm_duration', 'distance_delta', 'duration_delta']]
# df_row_check['duration'] = df_row_check['duration']/60
# df_row_check['osrm_duration'] = df_row_check['osrm_duration']/60 
# df_row_check['duration_delta'] = df_row_check['duration_delta']/60 